## Modelling Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Modelling of the zero coupon bond forward rates using observable market data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import CubicSpline
from scipy.optimize import minimize

## Section 1 - Load Inputs

In [0]:
# Set as a parameter here for now
as_of_date = '2025-06-30'

In [0]:
# Read cleaned and enriched inputs from silver layer
df_yields = spark.table("workspace.riskfree_silver.003_rbnz_yields_transformed") \
    .filter(f"(date = '{as_of_date}' AND series = 'Official Cash Rate (OCR)' AND group = 'Cash rate') OR (date = '{as_of_date}' AND series NOT LIKE '%year%' AND group = 'Secondary market government bond yields')")
df_amounts   = spark.table("workspace.riskfree_silver.002_nzdm_govtbonds_onissue_enriched").filter(f"as_of_date = '{as_of_date}' AND group = 'Secondary market government bond yields'")

In [0]:
display(df_yields)
display(df_amounts)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

## Section 2 - Pre-processing of data

In [0]:
# Convert spark to pandas for modelling
pdf_yields = df_yields.toPandas()
pdf_amounts = df_amounts.toPandas()

# Merge the dataframes based on series_id
merged_pdf = pdf_yields.merge(pdf_amounts, on='series_id', how='left')

## Section 3 - Bootstrapping of zero-coupon forward rates

The one-month forward rate is determined from the one-month Treasury bill.

Nominal Government bonds are decomposed into maturity and individual coupon payments to produce a set of equivalent zero-coupon nominal bonds maturing on the 15th of the month

A forward rate is determined for the shortest nominal Government bond, for the period up until the first nominal bond matures. For the period between the first nominal bond and the nominal second bond a forward rate is determined so that the second nominal bond market value is equalled using the previous forward rates as well. This process is repeated to solve for each successive forward rate until all nominal bonds have been valued.

In [0]:
# Sort by term_yr and keep only values needed
df = merged_pdf.sort_values(by='term_yr')[["term_yr", "yield_decimal"]].rename(columns={"yield_decimal": "spot_rate_pa"})

# Discount Factors from spot rates
df["discount_factor"] = 1 / (1 + df["spot_rate_pa"]) ** df["term_yr"]

# Bootstrap forward rates between adjacent terms
fwd_rates = []
for i in range(1, len(df)):
    t0 = df.loc[i-1, "term_yr"]
    t1 = df.loc[i, "term_yr"]
    D0 = df.loc[i-1, "discount_factor"]
    D1 = df.loc[i, "discount_factor"]
    fwd = (D0 / D1) ** (1 / (t1 - t0)) - 1
    fwd_rates.append((t0, t1, fwd))

# Final forward rate DataFrame
df_fwd = pd.DataFrame(fwd_rates, columns=["from_year", "term_yr", "fwd_rate_bootstrapped"])[["term_yr", "fwd_rate_bootstrapped"]]
df_bootstrapped = df.merge(df_fwd, on="term_yr", how="left")[["term_yr", "spot_rate_pa", "fwd_rate_bootstrapped"]].fillna({"fwd_rate_bootstrapped": df["spot_rate_pa"]})

display(df_bootstrapped)

In [0]:
# Plotting
plt.figure(figsize=(15, 6))
plt.scatter(df_bootstrapped["term_yr"], df_bootstrapped["spot_rate_pa"], color='blue', label='Spot Rate (pa)')
plt.step(df_bootstrapped["term_yr"], df_bootstrapped["fwd_rate_bootstrapped"], where='post', color='red', label='Forward Rate (bootstrapped)')

plt.xlabel('Term Year')
plt.ylabel('Rate')
plt.title('Spot Rates and Bootstrapped Forward Rates')
plt.legend()
plt.grid(True)
plt.show()

## Section 4 - Curve fitting and interpolation

The process is to fit a curve of forward rates to the zero-coupon portfolio of available bonds. The parameters of the fitted curve are determined by solving to minimize the least squares differences of the resulting fitted spot rates with the actual market spot rates. Two-, three- and six-month Treasury bill rates are used in addition to nominal Government bonds.

Market yields are weighted by the lesser of the amount available in the market, which excludes the amounts held by the Reserve Bank of New Zealand (RBNZ) and the Earthquake Commission (which is not usually traded) and $4 billion. This means that implied forward rates automatically give less weight to those bonds which represent a smaller proportion of the tradeable market.

The curve fitted is a cubic spline on the forward rates with 4 knots. This is fairly standard methodology with enough flexibility to fit most yield curves. There is some judgment involved in selecting the position of the knots, but this also gives a little flexibility to cope with any anomalies that may be present in the yield curve without changing the fundamental principles.

In [0]:
# Sort by term_yr and keep only values needed
df = merged_pdf.sort_values(by='term_yr')[["term_yr", "yield_decimal", "market_bonds_m"]].rename(columns={"yield_decimal": "spot_rate_pa"})

# Set a default high weight to T-bills and OCR with no weight
df["weight"] = np.where(
    df["market_bonds_m"].isnull(),
    4e4,
    np.minimum(df["market_bonds_m"], 4e9)
)

# Exclude OCR
df = df[df["term_yr"] > 0]

df

Objective Function: Minimize Weighted Squared Errors

In [0]:
# We'll fit forward rates at those knots, and interpolate in between
def fit_forward_curve(knot_fwd_rates):
    # Build cubic spline forward curve
    spline = CubicSpline(knots, knot_fwd_rates, bc_type='natural', extrapolate=True)
    
    # Calculate model discount factors from forward rates
    times = df["term_yr"].values
    fwd_vals = spline(times)

    # Prevent negative rates
    fwd_vals = np.clip(fwd_vals, 0.0001, None)

    discount_factors = np.exp(-np.cumsum(fwd_vals * np.gradient(times)))  # Numerical integral

    # Model spot rate = ln(1 / D(t)) / t
    spot_model = -np.log(discount_factors) / times

    # Weighted squared error
    spot_market = df["spot_rate_pa"].values
    weights = df["weight"].values
    error = np.sum(weights * (spot_model - spot_market) ** 2)

    return error

Solve for Knot Parameters

In [0]:
# Choose 1–2 fixed anchor knots for stability
base_knots = [0.083, 1.0]

# Add quantile-based knots for adaptiveness
quantile_knots = np.quantile(df["term_yr"], [0.5, 0.9])

knots = np.sort(np.concatenate([base_knots, quantile_knots]))
knots

In [0]:
# Initial guess: flat forward rate
init_guess = np.full(len(knots), 0.02)

# Minimize the weighted squared error between model and market spot rates
res = minimize(fit_forward_curve, init_guess, method="L-BFGS-B", bounds=[(0.0001, 0.1)] * len(knots))

# Extract the fitted forward rates at the knot points
fitted_knot_rates = res.x

fitted_knot_rates

Output Full Forward Curve

In [0]:
fwd_curve = CubicSpline(knots, fitted_knot_rates, bc_type='natural', extrapolate=True)

# Generate dense output
max_fit_term = df["term_yr"].max()
output_terms = np.arange(0.1, max_fit_term + 0.25, 0.25)

fwd_rates = fwd_curve(output_terms)
fwd_rates = np.clip(fwd_rates, 0.0001, 0.055)

# Compute discount factors and spot rates
delta = np.gradient(output_terms)
discount_factors = np.exp(-np.cumsum(fwd_rates * delta))
spot_rates = -np.log(discount_factors) / output_terms

df_cubic = pd.DataFrame({
    "term_yr": output_terms,
    "fwd_rate_cubic": fwd_rates,
    "spot_rates_pa_cubic": spot_rates
})

df_all = pd.merge_asof(df_cubic, df_bootstrapped, on="term_yr", direction="nearest")

df_all = df_all.sort_values(by="term_yr").bfill()

display(df_all)

Databricks visualization. Run in Databricks to view.

## Section 5 - Bridging to long-term riskfree rate

Bridging is required from the last observable market data point, out to a long-term assumption. The methodology applies linear interpolation over 10 years from the maturity date of
the last nominal Government bond, subject to a maximum slope of 0.05% pa.

In [0]:
# --- Step 1: Identify last bond maturity ---
T_last = df["term_yr"].max()  # Max maturity in your data, e.g. 20 or 25
long_term_rate = 0.048
max_slope = 0.0005  # 0.05% per year

# --- Step 2: Get last forward rate value before T_last ---
last_fwd_rate = df_all[df_all["term_yr"] <= T_last]["fwd_rate_cubic"].iloc[-1]

# Calculate required years to ramp at max slope
required_years = abs(long_term_rate - last_fwd_rate) / max_slope
T_extension = max(10, np.ceil(required_years * 4) / 4)  # Round up to nearest 0.25
T_max = T_last + T_extension

# --- Step 3: Compute capped slope ---
raw_slope = (long_term_rate - last_fwd_rate) / T_extension
if raw_slope >= 0:
    slope = min(raw_slope, max_slope)
else:
    slope = max(raw_slope, -max_slope)

# --- Step 4: Build extension terms and forward rates ---
extension_terms = np.arange(T_last + 0.25, T_max + 0.25, 0.25)
extension_fwd_rates = last_fwd_rate + slope * (extension_terms - T_last)

# --- Step 5: Combine with original forward curve ---
full_terms = np.concatenate([df_all["term_yr"], extension_terms])
full_fwd_rates = np.concatenate([df_all["fwd_rate_cubic"], extension_fwd_rates])

# Expand to 100 years at 0.25 intervals
full_terms = np.arange(0.1, 100.25, 0.25)

# Forward fill the forward rates
full_fwd_rates = np.interp(full_terms, np.concatenate([df_all["term_yr"], extension_terms]), np.concatenate([df_all["fwd_rate_cubic"], extension_fwd_rates]))

# --- Step 6: Compute spot rates from extended forward curve ---
delta = np.gradient(full_terms)
discount_factors = np.exp(-np.cumsum(full_fwd_rates * delta))
spot_rates = -np.log(discount_factors) / full_terms

# --- Step 7: Final output ---
df_extended = pd.DataFrame({
    "term_yr": full_terms,
    "fwd_rate_extended": full_fwd_rates,
    "spot_rate_extended": spot_rates
})

display(df_extended)

Databricks visualization. Run in Databricks to view.